Para este desdafío se utiliza Python 3.7.3 y las siguientes librerías:
- numpy==1.19.1
- pandas==1.2.4
- matplotlib==3.3.1 
- scikit-learn==0.23.2
- statsmodels=0.12.0 

In [ ]:
# Cargamos librerías estándar que usaremos en esta primera parte del desafío
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Datos:
    
Cargamos los datos usando la librería Pandas y visualizamos para hacer una rápida inspección visual.    
  

In [ ]:
# Cargamos datos de precipitaciones
precip = pd.read_csv('precipitaciones.csv')

# Mostramos los datos de precipitaciones para una rápida inspección visual
precip.head()

In [ ]:
# Cargamos datos del banco central
banco = pd.read_csv('banco_central.csv')

# Mostramos los datos del banco central para una rápida inspección visual
banco.head()

# 2. Análisis de los datos

Al cargar los datos, se ve que hay errores e inconsistencias en los formatos usados. Por ejemplo, hay ambigüedades respecto a cómo se deben interpretar los puntos en datos numéricos.

En los datos del banco central hay datos duplicados, valores que no numéricos (hay unas celdas con valor 'a') y muchos datos faltantes, que son rellenados con 'Nan'. 

Adicionalmente, los las fechas de los datos del banco central incluyen horas, minutos y segundo (aunque no es información relevante en este caso). Los datos de precipicationes solo incluyen la fecha.

Para cada una de las tablas de datos estandarizamos el uso de la fecha, eliminamos columnas vacías, removemos duplicados, convertimos valores a numéricos y eliminamos outliers (consideramos outliers datos bajo el cuantil 0.01 y sobre a 0.99). Aunque no haya outliers, se eliminará el 2% de los datos de los extremos de la distribución lo que no debería impactar mucho el análisis posterior. Un análisis más detallado se podría hacer para solamente aplicar este critero a los datos más 'ruidosos', pero tomaría más tiempo.

In [ ]:
# Convertimos los datos de la columna 'date' a datetime
precip['date'] = pd.to_datetime(precip['date'], errors='coerce', yearfirst=True).dt.date
# Seteamos el datetime como el index de la tabla
precip = precip.set_index('date')
# Eliminamos filas con index inválido (Nan). Al aprecer en esta tabla no hay, pero es mejor asegurarse. 
precip = precip[precip.index.notnull()]
# Si hay datos duplicados, los eliminamos. Al aprecer en esta tabla no hay, pero es mejor asegurarse. 
precip = precip.drop_duplicates()
# Por último convertimos los valores de las céldas a numérico. 
# Se puede apreciar que los puntos son usados para indicar miles y no decimales, lo que no es interpretado por Pandas.
# Simplemente removemos los puntos y convertimos a numérico. Usamos 'coerce' para forzar inválidos a 'Nan'
for col_name in precip:
    precip[col_name] = [x.replace('.','') if isinstance(x, str) else x for x in precip[col_name] ]    
    precip[col_name] = pd.to_numeric(precip[col_name] ,errors='coerce')
# Removemos posibles outliers, es decir datos bajo el cuantil 0.01 y sobre 0.99
precip = precip[(precip < precip.quantile(0.99)) & (precip>  precip.quantile(0.01))]
# Ordenamos por fecha
precip = precip.sort_index()

# Nuevamente mostramos los datos para verificar que la tabla está con el formato deseado
precip

In [ ]:
# Convertimos los datos de la columna 'Periodo' a datetime y eliminamos la hora
banco['Periodo'] = pd.to_datetime(banco['Periodo'], errors='coerce', yearfirst=True).dt.date
# Cambiamos el nombre de la columna 'Periodo' a 'date' para que sea consistente con las fechas de precipitaciones
banco = banco.rename({'Periodo': 'date'}, axis=1)
# Seteamos el datetime como el index de la tabla
banco = banco.set_index('date')
# Eliminamos filas con index inválido (Nan)
banco = banco[banco.index.notnull()]
# Hay datos duplicados, los eliminamos
banco = banco.drop_duplicates()
# Por último convertimos los valores de las céldas a numérico. 
# Se puede apreciar que los puntos son usados para indicar miles y no decimales, lo que no es interpretado por Pandas.
# Simplemente removemos los puntos y convertimos a numérico. Usamos 'coerce' para forzar inválidos a 'Nan'
for col_name in banco:
    banco[col_name] = [x.replace('.','') if isinstance(x, str) else x for x in banco[col_name] ]    
    banco[col_name] = pd.to_numeric(banco[col_name] ,errors='coerce')
# Removemos posibles outliers, es decir datos bajo el cuantil 0.01 y sobre 0.99
banco = banco[(banco < banco.quantile(0.99)) & (banco >  banco.quantile(0.01))]
# Ordenamos por fecha
banco = banco.sort_index()

# Nuevamente mostramos los datos para verificar que la tabla está con el formato deseado
banco

# 3. Visualización

Creamos una función para filtrar dats de precipitaciones de una región en el rango de fechas deseado. En Pandas se puede filtrar facilmente por rangos de fechas!

In [ ]:
def filt_precip(data, region, start_date, end_date):
    """
        data: pd.DataFrame
        region: str
        start_date: str or datetime.date
        end_date: str or datetime.date
    """
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date).date()
    if isinstance(end_date, str):    
        end_date = pd.to_datetime(end_date).date()
    
    # Nos aseguramos que la región sea válida
    assert region in data.columns, "Región inválida."
    # Nos aseguramos que la fecha de inicio sea válida    
    assert start_date in data.index, "Fecha de inicio inválida."
    # Nos aseguramos que la fecha de término sea válida    
    assert end_date in data.index, "Fecha de término inválida."  
    # Nos aseguramos que la fecha de inicio sea menor a la de término
    assert start_date < end_date, "Fecha de inicio es posterior a la de término"  
    # Filtramos entre las fechas dadas
    return data[region][(data.index>start_date) & (data.index<end_date)]  

# Función para calcular la media móvil con una ventana deseada
def moving_average(x, w):
    """
        x: list or np.array
        w: int or float
    """
    return np.convolve(x, np.ones(w), 'same') / w

Obtenemos los datos para la región metropolitana en las fechas indicadas. Claramente se observa la tendencia estacional de aumentar la cantidad de precipitaciones una vez al año. Se puede apreciar como la media móvil ha ido disminuyendo en las ultimas dos décadas...

In [ ]:
# Creamos variables con rangos de fecha
start_date = '2000-01-01'
end_date = '2020-01-01'

# Filtramos Datos
filt_rm = filt_precip(precip, 'Metropolitana_de_Santiago',start_date, end_date)

filt_rm.plot(label='precipitaciones')
# tambien graficamos la media móvil con una ventana de 10 
ma_rm = moving_average(filt_rm, 10)
plt.plot(filt_rm.index,ma_rm, label='media móvil')
plt.xlabel('Fecha')
plt.ylabel('precipitación [mm]')
plt.title('Precipitaciones RM entre {}:{}'.format(start_date, end_date))
plt.legend()
plt.show()

In [ ]:
# Filtramos datos
filt_vi =  filt_precip(precip, 'Libertador_Gral__Bernardo_O_Higgins',start_date, end_date)

filt_vi.plot(label='precipitaciones')
ma_vi = moving_average(filt_vi, 10)
plt.plot(filt_vi.index,ma_vi, label='media móvil')
plt.xlabel('Fecha')
plt.ylabel('precipitación [mm]')
plt.title('Precipitaciones VI entre {}:{}'.format(start_date, end_date))
plt.legend()
plt.show()

Ahora creamos una función para graficar una lista de años de una región en particular

In [ ]:
# Lista con los nombres de cada mes
month_names = ['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']

def plot_region_by_years(data, region, years):
    """
        data: pd.DataFrame
        region: str
        years: list
    """
    
    # Nos aseguramos que la región sea válida
    assert region in data.columns, "Region inválida."
    
    # Filtramos la región de interés
    data = data[region]
    
    # Creamos dict para almacenar data de los años de interés
    plot_data = {}
    for year in years:
        # Cada valor del dict tendrá una lista con el año y el valor
        plot_data[year] = []
        for curr_date in data.index:
            if curr_date.year == int(year):
                plot_data[year].append([curr_date.month, data.loc[curr_date]])

    # Creamos objeto para graficar
    f, ax = plt.subplots()
    for year, values in plot_data.items():
        # ordenamos por mes los datos de cada año
        sorted(values)
        # Convertimos índice de cada mes a su nombre
        months = [month_names[x[0]-1] for x in values]
        values = [x[1] for x in values]
        # Graficamos el año correspondiente
        ax.plot(months, values, label=year)
    ax.legend()
    ax.set_ylabel('Precipitaciones [mm]')
    ax.set_title('Precipitaciones de {} para los años {}'.format(region, years))
    return ax

    

Se puede apreciar como la cantidad de lluvias ha disminuído sustancialmente a lo largo de los años. Si bien hay datos faltantes de 1982, la tendendencia es clara y de sostenida en el tiempo.

In [ ]:
plot_region_by_years(precip, 'Maule',[1982, 1992, 2002, 2012, 2019])

Creamos una función para graficar series históricas de PIB en un determinado rango de fechas. La verdad es que esta función hace lo mismo que la ya creada anteriormente (*filt_precip*), solamente que ahora debe recibir dos campos. Creamos una función similar a la anterior, pero adaptada.

In [ ]:
def filt_pibs(data, pib1, pib2, start_date, end_date):
    """
        data: pd.DataFrame
        pibs: list
        start_date: str or datetime.date
        end_date: str or datetime.date
    """
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date).date()
    if isinstance(end_date, str):    
        end_date = pd.to_datetime(end_date).date()
    
    # Nos aseguramos que la los campos de pib sea válidos
    assert pib1 in data.columns , "Campo inválido."
    assert pib2 in data.columns, "Campo inválido."    
    # Nos aseguramos que la fecha de inicio sea válida    
    assert start_date in data.index, "Fecha de inicio inválida."
    # Nos aseguramos que la fecha de término sea válida    
    assert end_date in data.index, "Fecha de término inválida."  
    # Nos aseguramos que la fecha de inicio sea menor a la de término
    assert start_date < end_date, "Fecha de inicio es posterior a la de término" 
    
    data = data[[pib1, pib2]]
    # Filtramos entre las fechas dadas
    return data[(data.index>start_date) & (data.index<end_date)]  


Graficamos los PIBs pedidos desde el año 2013 en adelante. Puesto que nuestras tablas están ordenadas por fecha, podemos buscar el último índice para tener la fecha más reciente.

Se puede ver que, aunque haya datos faltantes, el PIB de servicios financieros ha ido aumentando consistentemente en los últimos años y no presenta variaciones estacionales. Por otra parte, el PIB agropecuario y silvícola tiene notorias alzas en los periodos de verano y fuertes caídas en invierno, por lo que claramente presenta una periodicidad relacionada con las estaciones del año.

Se aprecia que desde el año 2013 hasta el 2018 ambos PIBs fueron aumentando en conjunto(si consideramos los periodos altos del agropecuario y silvícola), pero la tendencia no se aprecia desde el 2019 a la fecha.

In [ ]:
filtered_pibs = filt_pibs(banco,'PIB_Agropecuario_silvicola',
                          'PIB_Servicios_financieros',
                          '2013-01-01',
                           banco.index[-1])
filtered_pibs.plot()


# 4. Tratamiento y creación de variables

Una forma clásica de evaluar la correlación entre dos variables es midiendo su coeficiente de correlación de Pearson. Esta entrega una matriz cuadrada y simétrica que indica que tan correlacionadas están las variables. 
Para entrenar cualquier tipo de modelo es necesario tener variables que estén correlacionadas (ya sea de manera positiva o negativa) con el/los valor/es objetivo. En caso de no haber correlación, o de ser muy pequeña, esos datos no serán útiles para ese problema de predicción/clasificación/regresión.

Cargamos los datos de la leche y creamos una variable 'date' a partir de el año y el mes.

In [ ]:
# Cargamos datos de la leche
leche = pd.read_csv('precio_leche.csv')

# Ya habíamos creado una variable month_names. Ahora les asignamos un índice en un diccionario
month_dict = {v:k for k,v in enumerate(month_names, start=1)}
# Mapeamos los nombres de cada mes a índice en la tabla de datos de leche
leche['Mes'] = leche['Mes'].map(month_dict)
# Creamos datos del día correspondiente a cada fecha
leche['Dia'] = [1]*len(leche.index)
# Creamos variable date a partir del año, mes y dia
leche['date'] = pd.to_datetime(dict(year=leche.Anio,month=leche.Mes,day=leche.Dia)).dt.date
# Eliminamos las columnas de 'Anio', 'Mes' y 'Dia' puesto que ya no son necesarias...
leche.drop(['Anio'], axis=1, inplace=True)
leche.drop(['Mes'], axis=1, inplace=True)
leche.drop(['Dia'], axis=1, inplace=True)


# Al igual que con las tablas anteriores, convertimos el date a indice
leche = leche.set_index('date')
# Eliminamos filas vacías y duplicados
leche = leche[leche.index.notnull()]
leche = leche.drop_duplicates()

Para realizar el merge entre las bases de datos hay un problema! No tienen los mismos rangos de fechas, por lo que creamos una función para agregar las fechas faltantes (y llenarlas con datos vacíos) entre las bases de datos.

In [ ]:
def synch_data(data1, data2, data3, fill=np.nan):
    """
        data1: pd.dataFrame
        data2: pd.dataFrame
        data3: pd.dataFrame
        fill: np.nan or int or float
    """
    # Funcion anidada auxiliar para agregar una nueva fila en la fecha indicada
    def fill_data(data, idx):
        data.loc[idx] = [fill]*len(data.columns)
        return data
    # Para cada base de datos comparamos las fechas faltantes en las otras y las agregamos.
    for d1 in data1.index:
        if d1 not in data2.index:
            data2 = fill_data(data2, d1)
        if d1 not in data3.index:
            data3 = fill_data(data3, d1)                    
    for d2 in data2.index:
        if d2 not in data1.index:
            data1 = fill_data(data1, d2) 
        if d2 not in data3.index:
            data3 = fill_data(data3, d2) 
    for d3 in data3.index:
        if d3 not in data1.index:
            data1 = fill_data(data1, d3)
        if d3 not in data2.index:
            data2 = fill_data(data2, d3)
    # Devolvemos los valores sincronizados y ordenados por fecha      
    return data1.sort_index(), data2.sort_index(), data3.sort_index()

In [ ]:
print('Tamaños de las bases de datos antes de sincronizar:\n  -precipitaciones:{}\n  -banco:{}\n  -leche:{}'.format(len(precip.index),len(banco.index),len(leche.index)))
precip, banco, leche = synch_data(precip, banco, leche)
print('Tamaños de las bases de datos después de sincronizar:\n  -precipitaciones:{}\n  -banco:{}\n  -leche:{}'.format(len(precip.index),len(banco.index),len(leche.index)))


Ahora que las bases de datos tienen las mismas fechas, podemos crear una base de datos que sea un *merge* de todas las bases de datos. Como sincronizamos los datos, ahora hay varias filas que contienen principalmente *Nans*

In [ ]:
merged_db = pd.concat([banco, precip, leche], axis=1)
merged_db.head()

# 5 Modelo

La idea es entrenar un modelo que prediga el valor futuro del precio de la leche en función de sus valores pasados y de valores pasados de otras variables externas. 

Para este tipo de problema hay varias alternativas, desde análisis clásico de series de tiempo basados en modelos autoregresivos hasta técnicas modernas de *deep learning*, por ejemplo, basadas en redes neuronales recurrentes. Dado que no se cuenta con un volumen tan grande de datos y que varios de estos están incompletos, la mejor alternativa es optar por un método sencillo.

Antes de elegir el método, investiguemos el comportamiento del precio de la leche:

In [ ]:
# Obtenemos un np.array con el precio de la leche
p_leche = merged_db['Precio_leche'].to_numpy()
# Indices de fecha
idx = merged_db.index

plt.plot(idx, p_leche, label='Precio')
ma_leche = moving_average(p_leche, 10)
plt.plot(idx, ma_leche, label='media móvil')
plt.title('Precio de la leche')
plt.ylabel('Precio [$]')
plt.xlabel('Date')
plt.legend()
plt.show()

Se puede apreciar que no es estacionario (ni si quiera en el sentido amplio) y que su valor ha ido aumentando con los años. Veamos que pasa con las diferencias de precio entre un año y el anterior:

In [ ]:
# Función para reemplazar Nans por la interpolación lineal de los valores vecinos que sean válidos 
def interpolate_nans(y):
    def nan_helper(yy):
        return np.isnan(yy), lambda z: z.nonzero()[0]
    nans, x= nan_helper(y)
    y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    return y

In [ ]:
# interpolamos los valores de los Nans
p_leche = interpolate_nans(p_leche)
# Obtenemos la derivada de los valores de la leche
diff_p_leche = p_leche[1:]-p_leche[:-1]

plt.plot(idx[1:],diff_p_leche, label='Precio')
ma_leche = moving_average(diff_p_leche, 10)
plt.plot(idx[1:], ma_leche, label='media móvil')
plt.title('Derivada de precios de la leche')
plt.ylabel('Precio [$]')
plt.xlabel('Date')
plt.legend()

Vemos que la diferencia de valores del precio de la leche es estacionario!


Por lo tanto usaremos un modelo auto regresivo integrado de promedio movil con variables exógenas (ARIMAX), en que el orden de la parte integrativa será 1. Las variables exógenas deben ser algunos datos del banco central y de las precipitaciones. Para elegir las variables externas, analizaremos los coeficientes de correlación de Pearson entre las variables y el precio de la leche, las que correlacionen mejor serán utilizadas. 

In [ ]:
# Nombres de las variables que no sean precio se la leche
var_names = merged_db.columns.drop('Precio_leche')
# Creamos diccionario que almacenará el coeficiente de correlación de cada variable con el precio de la leche:
var_corr = {i:None for i in var_names}

# iteramos sobre cada variable y calculamos la correlacion
for var_name in var_names:
    # Extraemos variable y pasamos a np.array
    curr_var = merged_db[var_name].to_numpy()
    # Inteprolamos Nans
    curr_var = interpolate_nans(curr_var)
    # Calculamos matriz de correlación. Matriz simétrica, las diagonales son varianzas normalizadas (no interesan)
    corr = np.corrcoef(curr_var, p_leche)[0][1]
    # Almacenamos en nuestro dict el valor absoluto (correlaciones negativas también sirven!)
    var_corr[var_name] = abs(corr)
    
# Hacemos un rank de las variables según el valor absoluto de la correlación
ranked_corrs = {k: v for k, v in sorted(var_corr.items(), key=lambda item: item[1])}
ranked_corrs

Los valores que tienen mayor correlación (positiva o negativa) son 'Precio_de_la_onza_troy_de_oro_dolaresoz', 'Precio_del_cobre_refinado_BML_dolareslibra' y 'Precio_de_la_onza_troy_de_plata_dolaresoz'!

Usaremos estas variables para entrenar el modelo ARIMAX. Utilizaremos la librería *statsmodels* para realizar el entrenamiento y en particular la clase SARIMAX, que es una versión del modelo ARIMAX.

In [ ]:
# Importamos librerías
import random 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

#Number of points
n_points = len(merged_db.index)
# Agregamos una dimension al precio de la leche
p_leche = p_leche.reshape(n_points,1)
# Obtenemos nuestras variables exógenas:
ex_data1 = interpolate_nans(merged_db['Precio_de_la_onza_troy_de_plata_dolaresoz'].to_numpy()).reshape(n_points,1)
ex_data2 = interpolate_nans(merged_db['Precio_del_cobre_refinado_BML_dolareslibra'].to_numpy()).reshape(n_points,1)
ex_data3 = interpolate_nans(merged_db['Precio_de_la_onza_troy_de_oro_dolaresoz'].to_numpy()).reshape(n_points,1)
ex_data = np.concatenate((ex_data1, ex_data2, ex_data3), axis=1)


Ahora debemos ver la forma de separar los datos para poder testear el modelo.
Los parámetros del modelo son mejor *fiteados* cuando usamos más datos, sin embargo estamos utilizando un modelo autoregresivo de orden 1. Esto implica que podemos tomar segmentos de tamaño fijo e intentar predecir el siguiente valor del precio de la leche, en base a eso obtener métricas de desempeño.

Realizaremos cortes aleatorios de segmentos de 100 valores temporales y mediremos la calidad de la predicción utilizando el error absoluto medio (MAE). Repetiremos este proceso 100 veces, por lo que estaremos entrenando 100 modelos en total! El proceso es rápido, por lo que no debiese demorar mucho.

Nota: Aproximadamente los primeros 100 valores del precio de la leche fueron interpolados, asi que no es muy informativo tratar de usarlos. Samplearemos índices desde el 100 en adelante


In [ ]:
# Obtenemos los índices de inicio de las sub-secuencias que muestrearemos
random.seed(1)
idx = random.sample(range(100, n_points-102), k=100)

# Lista de valores objetivo reales
targets = []
# lista de valores predichos
predictions = []

for i in idx:
    curr_p_leche = p_leche[i:i+100]
    curr_ex_data = ex_data[i:i+100,:]
    
    model = SARIMAX(curr_p_leche, exog=curr_ex_data, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0))
    fitted_model = model.fit()
    pred = fitted_model.forecast(exog=ex_data[i+10,:])    
    
    targets.append(p_leche[i+101])
    predictions.append(pred)

mae = mean_absolute_error(targets, predictions)   
print('El MAE de nuestro modelo es:', mae)

In [ ]:
plt.plot(targets,label='Objetivo')
plt.plot(predictions,label='predicción')
plt.legend()
plt.xlabel('Unidades temporales')
plt.ylabel('Precio de la leche')
plt.title('Predicción del precio de la leche, MAE:{0:.2f}'.format(mae))
plt.show()

El MAE de la predicción es 6.32, lo cual es bueno. Entre los datos proporcionados hay variables que correlacionan bastante bien con el precio de la leche y se puede apreciar en el gráfico que los valores suelen ser muy similares. Se podría investigar cómo varía el desempeño del modelo agregando otras variables exógenas que correlacionen bien con el precio de la leche, pero habría que investigarlo.

Sería bueno tener los datos faltantes, que actualmente fueron interpolados linealmente, para poder aumentar el volumen de datos y tener predicciones más fidedignas. 

Este tipo de problemas de predicción utilizando series de tiempo se evalúan midiendo el error entre el valor predicho y el real. En nuestro caso consideramos el valor absoluto medio, pero también se podría utilizar el error cuadrático medio (MSE) o su raíz cuadrada (RMSE).

Este tipo de modelos predictivos son muy útiles cuando se tiene series de tiempo que correlacionen bien con algun valor a predecir y si existen suficientes datos, se pueden armar modelos más sofisticados que son capaces de obtener muy buenos resultados. En particular se pueden utilizar para predecir características metereológicas, como la cantidad de lluvia que caerá o cuánto sol habrá cada día. Con este tipo de modelos se puede optimizar, por ejemplo, la generación de energías no renovables (si predecimos la potencia del viento o la intensidad del sol, sabemos cuánta energía renovable tendremos) y así no desperdiciar ni contaminar de manera innecesaria. Similarmente, se puede utilizar para tomar decisiones en la agricultura y hacer un uso más eficiente de las aguas o decidir que tipo de árboles plantar para las siguientes temporadas. Estas aplicaciones tienen un efecto directo en el cambio climático, puesto que ayudarían a disminuir contaminación y emisiones no necesarias.

